In [1]:
import pickle
import numpy as np
import pandas as pd
import time

In [55]:
raw_data['signal']['wrist'].keys()

dict_keys(['ACC', 'BVP', 'EDA', 'TEMP'])

In [128]:
data_path = "/Users/alaa/data/WESAD"
nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
data_list, cnt = collate_wrist_data(data_path, nums)

S2 : 24316 rows, 8484 measurements
S3 : 25972 rows, 8620 measurements
S4 : 25692 rows, 8660 measurements
S5 : 25032 rows, 8868 measurements
S6 : 28284 rows, 8808 measurements
S7 : 20952 rows, 8792 measurements
S8 : 21864 rows, 8836 measurements
S9 : 20892 rows, 8788 measurements
S10 : 21984 rows, 9108 measurements
S11 : 20932 rows, 8912 measurements
S13 : 22148 rows, 8904 measurements
S14 : 22192 rows, 8908 measurements
S15 : 21008 rows, 8932 measurements
S16 : 22524 rows, 8884 measurements
S17 : 23680 rows, 9104 measurements
Total Num of Measurements: 132608


In [129]:
for df, idx in zip(data_list, nums):
    df.to_csv(f"S{idx}.csv")

In [120]:
label = [1, 2, 3]
stacked_df = (data_list[0])
stacked_df["subject"] = "S_2"
stacked_df = stacked_df.loc[stacked_df.label.isin([1, 2, 3])].reset_index()
i = 1
for j in nums[1:]:
    d = data_list[i]
    d['subject'] = f"S_{j}"
    d = d.loc[d.label.isin([1, 2, 3])].reset_index()
    stacked_df = pd.concat([stacked_df, d])
    i += 1 

    

In [124]:
stacked_df.reset_index(drop=True, inplace=True)

In [125]:
stacked_df

,index,label,ACC_0,ACC_1,ACC_2,BVP,EDA,TEMP,subject
0,306.75,1,65.0,12.0,21.0,-33.58,1.640539,35.81,S_2
1,307.00,1,55.0,-32.0,44.0,30.52,1.634132,35.81,S_2
2,307.25,1,49.0,-21.0,30.0,52.54,1.614912,35.81,S_2
3,307.50,1,52.0,-22.0,27.0,94.31,1.591848,35.81,S_2
4,307.75,1,58.0,-14.0,28.0,-38.86,1.558534,35.81,S_2
...,...,...,...,...,...,...,...,...,...
132603,4234.00,2,-100.0,-32.0,-21.0,-57.34,1.018841,32.66,S_17
132604,4234.25,2,-75.0,2.0,0.0,11.54,1.004767,32.66,S_17
132605,4234.50,2,-35.0,-3.0,25.0,-60.52,1.008606,32.66,S_17
132606,4234.75,2,-46.0,-5.0,51.0,-3.41,1.046988,32.66,S_17


In [127]:
data_list[0]

,label,ACC_0,ACC_1,ACC_2,BVP,EDA,TEMP,subject
0.00,0,62.0,-21.0,107.0,-59.37,1.138257,35.41,S_2
0.25,0,51.0,16.0,35.0,43.50,1.125444,35.41,S_2
0.50,0,53.0,21.0,-6.0,53.56,1.011405,35.41,S_2
0.75,0,55.0,17.0,34.0,54.64,1.033188,35.41,S_2
1.00,0,48.0,24.0,15.0,-117.88,0.935807,35.41,S_2
...,...,...,...,...,...,...,...,...
6077.75,0,47.0,14.0,41.0,27.06,0.063052,34.21,S_2
6078.00,0,46.0,15.0,40.0,14.01,0.060489,34.23,S_2
6078.25,0,49.0,15.0,41.0,-0.43,0.059208,34.23,S_2
6078.50,0,46.0,17.0,36.0,-32.72,0.073303,34.23,S_2


In [121]:
def clean_df(labels=[1,2,3]):
    cnts = []
    grouped = df.groupby('label')
    for l in labels:
        l_cnt = grouped.get_group(l).shape[0]
        cnts.append(l_cnt)
    return cnts

In [89]:
def collate_wrist_data(data_path, subject_ids, labels=[1, 2, 3]):
    data_list = []
    total_label_cnts = []
    for sub_id in subject_ids:
        fname = f"{data_path}/S{sub_id}/S{sub_id}.pkl"
        raw_data = load_data(fname)
        data = process_data(raw_data)
        label_cnt = count_labels(data, labels)
        total_label_cnts.append(sum(label_cnt))
        data_list.append(data)
        print(f'S{sub_id} : {data.shape[0]} rows, {sum(label_cnt)} measurements')
    print(f'Total Num of Measurements: {sum(total_label_cnts)}')
    return data_list, total_label_cnts

In [58]:
def count_labels(df, labels):
    cnts = []
    grouped = df.groupby('label')
    for l in labels:
        l_cnt = grouped.get_group(l).shape[0]
        cnts.append(l_cnt)
    return cnts

In [72]:
raw_data['signal']['wrist']['ACC'][:, 0].shape

(189440,)

In [82]:
def process_data(raw_data):
    freqs = {'label': 700, 'TEMP': 4, 'ACC': 32, 'BVP': 64, 'EDA': 4}
    wrist_data = raw_data['signal']['wrist']
    signal_keys = wrist_data.keys()
    df = pd.DataFrame(raw_data['label'], columns=['label'])
    df.index = [(1 / freqs['label']) * i for i in range(len(df))]
    for mod in signal_keys:
        if mod=='ACC':
            for i in range(3):
                df2 = pd.DataFrame(wrist_data[mod][:, i], 
                                   columns=[mod+f'_{i}'])
                df2.index = [(1 / freqs[mod]) * i for i in range(len(df2))]
                df = df.join(df2, how='inner')
        else:
            df2 = pd.DataFrame(wrist_data[mod], columns=[mod])
            df2.index = [(1 / freqs[mod]) * i for i in range(len(df2))]
            df = df.join(df2, how='inner')
    return df